# Test for Cl in the full sky case

In [ ]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from qubic import QubicSkySim as qss
from qubic import camb_interface as qc
%matplotlib inline
rc('font', size=15)
rc('figure', figsize=(13, 8))

In [ ]:
ell, totCL, unlensedCL = qc.get_camb_Dl(lmax=2500, r=0.1)

In [ ]:
rc('figure', figsize=(13, 8))
order = ['TT', 'EE', 'BB', 'TE']
for i in range(4):
    subplot(2,2,i+1)
    plot(ell,totCL[:,i], color='k', label='Total Cl')
    xlabel('$\\ell$')
    ylabel('$D_\\ell$')
    title(order[i])
    legend(loc='upper right')
tight_layout()

In [ ]:
# nside = 256
# mycls = np.zeros_like(totCL)
# for i in range(4):
#     mycls[2:, i] = 2 * np.pi * totCL[2:,i] / (ell[2:] * (ell[2:] + 1))
# maps=hp.synfast(mycls.T,nside, new=True)
# hp.mollview(maps[0,:], title="I map from synfast", sub=(2,3,1))
# hp.mollview(maps[1,:], title="Q map from synfast", sub=(2,3,2))
# hp.mollview(maps[2,:], title="U map from synfast", sub=(2,3,3))

In [ ]:
from qubicpack.utilities import Qubic_DataDir
import qubic
from qubic import QubicSkySim as qss
#### Create sample CMB I,Q,U maps
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])
dictfilename = global_dir + '/dicts/test_QubicSkySim.dict'

# Read dictionary chosen
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
nside = d['nside']
# Make a sky using PYSM
seed = None
cmb_dict = {'CAMBSpectra':totCL, 'ell':ell, 'seed':None}
sky_config = {'cmb': cmb_dict}
Qubic_sky = qss.Qubic_sky(sky_config, d)
x0 = np.reshape(Qubic_sky.get_simple_sky_map(),(d['nf_sub'],d['nside']**2*12,3))

print('Input Map with shape (nf_sub, #pixels, #stokes) : ', np.shape(x0))
rc('figure', figsize=(16, 10))
figure()
stn = ['I','Q','U']
rng = (np.std(x0, axis=(0,1))*3).astype(int)
rng[1:] = np.max(rng[1:])
numsub = 4
for i in range(d['nf_sub']):
    for istokes in [0,1,2]:
        hp.mollview(x0[i,:,istokes], min=-rng[istokes], max=rng[istokes],
                    sub=(d['nf_sub']+1,3,numsub), title=stn[istokes]+' subfreq {}'.format(i))
        numsub += 1
        
#### We take just the lower frequency map
maps = x0[0,:,:].T.copy()

In [ ]:
anafast_cl = hp.anafast(maps)
ll = np.arange(len(anafast_cl[0,:]))

In [ ]:
order = ['TT', 'EE', 'BB', 'TE']
for i in range(3):
    subplot(2,3,i+1)
    plot(ell,totCL[:,i], color='r', label='Total Cl')
    plot(ll, ll*(ll+1)/2/np.pi*anafast_cl[i,:],'o', alpha=0.1, label='Anafast')
    xlabel('$\\ell$')
    ylabel('$D_\\ell$')
    title(order[i])
    legend(loc='upper right')
    xlim(0,3*nside)
tight_layout()